In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import cross_validate

# Read data and choose to keep top 5 most complaint products

In [2]:
csv_file = 'D:/Jobs/If insurance/complaints.csv'
df = pd.read_csv(csv_file)

df = df[['Product', 'Consumer complaint narrative']]
df = df[pd.notnull(df['Consumer complaint narrative'])]

top_5 = df.groupby('Product').count().sort_values(by=['Consumer complaint narrative'], ascending=False).head(5)
#sample = top_5.values[4][0]
top_5 = list(top_5.index)

top_5_products = []
sample = 10000
for product in top_5:
    if(len(top_5_products) == 0):
        product = np.where((df['Product'] == product).values)[0]
        top_5_products = np.random.choice(product, sample, replace=False)
    else:
        product = np.where((df['Product'] == product).values)[0]
        top_5_products = np.concatenate((top_5_products, np.random.choice(product, sample, replace=False)))
df = df.iloc[top_5_products]
df['class'] = df['Product'].factorize()[0]

C:\Users\Huyen Vu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df

,Product,Consumer complaint narrative,class
1428259,"Credit reporting, credit repair services, or o...",Experian continually has my credit report cont...,0
234678,"Credit reporting, credit repair services, or o...",I sent a letter on XX/XX/XXXX to remove errone...,0
2154129,"Credit reporting, credit repair services, or o...",I received a hard inquiry XX/XX/XXXX When I ha...,0
2379882,"Credit reporting, credit repair services, or o...",I am not an attorney nor a third party I am fi...,0
1943252,"Credit reporting, credit repair services, or o...",I am writing to delete the following informati...,0
...,...,...,...
1100151,Checking or savings account,"I, XXXX XXXX XXXX, do not agree with the resol...",4
544774,Checking or savings account,"I opened a business checking account online, a...",4
2129437,Checking or savings account,On XX/XX/2018 I called my bank Capital One 360...,4
300991,Checking or savings account,"To whom it may concern, I have been banking wi...",4


# Convert a collection of raw documents to a matrix of TF-IDF features

In [4]:
tfidf = TfidfVectorizer()

# We transform each complaint into a vector
x = tfidf.fit_transform(df['Consumer complaint narrative']).toarray()

y = df['class']

In [5]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
accuracy = np.mean(cross_validate(model, x, y, cv=5)['test_score'])
print('accuracy is: ' + str(accuracy))

accuracy is: 0.7513


In [6]:
from sklearn.svm import LinearSVC
model = LinearSVC()
accuracy = np.mean(cross_validate(model, x, y, cv=5)['test_score'])
print('accuracy is: ' + str(accuracy))

accuracy is: 0.8739399999999999


In [7]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
accuracy = np.mean(cross_validate(model, x, y, cv=5)['test_score'])
print('accuracy is: ' + str(accuracy))

accuracy is: 0.8392200000000001


In [8]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()
accuracy = np.mean(cross_validate(model, x, y, cv=5)['test_score'])
print('accuracy is: ' + str(accuracy))

accuracy is: 0.87414


In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(alpha=1, max_iter=1000)
accuracy = np.mean(cross_validate(model, x, y, cv=5)['test_score'])
print('accuracy is: ' + str(accuracy))

In [10]:
X_train, X_test, y_train, y_test,indices_train,indices_test = train_test_split(x, y, df.index, test_size=0.2, random_state=1)
model = SGDClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_pred, target_names= df['Product'].unique()))

                                                                              precision    recall  f1-score   support

Credit reporting, credit repair services, or other personal consumer reports       0.85      0.84      0.85      2086
                                                             Debt collection       0.87      0.85      0.86      2001
                                                                    Mortgage       0.93      0.97      0.95      1951
                                                 Credit card or prepaid card       0.87      0.82      0.84      1983
                                                 Checking or savings account       0.87      0.93      0.90      1979

                                                                    accuracy                           0.88     10000
                                                                   macro avg       0.88      0.88      0.88     10000
                                                      